In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import pandas as pd

pubs = pd.read_csv("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Final_Filtered/Final_Authors_Topics_Filtered_Publications.csv")
authors = pd.read_csv("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Authors List/combined_authors_metadata.csv")
authors_order = pd.read_csv("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Authors List/authors_order_0.csv")

print(len(pubs))
print(len(authors))

12154
97647


In [128]:
northern_countries = ['algeria', 'egypt', 'libya', 'morocco', 'sudan', 'tunisia', 'western sahara']
southern_countries = ['angola', 'botswana', 'lesotho', 'mozambique', 'namibia', 'south africa', 'swaziland', 'eswatini', 'zambia', 'zimbabwe']
west_countries = ['benin', 'burkina faso', 'cape verde', 'cabo verde', "côte d'ivoire", "cote d'ivoire", 'ivory coast', 'gambia', 'ghana', 'guinea', 'guinea-bissau', 'liberia', 'mali', 'mauritania', 'niger', 'nigeria', 'senegal', 'sierra leone', 'togo', 'saint helena']
east_countries = ['burundi', 'comoros', 'djibouti', 'eritrea', 'ethiopia', 'kenya', 'madagascar', 'malawi', 'mauritius', 'réunion', 'reunion', 'rwanda', 'seychelles', 'somalia', 'somaliland', 'tanzania', 'united republic of tanzania', 'uganda', 'south sudan', 'mayotte']
central_countries = ['cameroon', 'central african republic', 'chad', 'congo', 'congo republic - brazzaville', 'congo-brazzaville', 'congo-kinshasa', 'republic of the congo', 'democratic republic congo', 'democratic republic of congo', 'democratic republic of the congo', 'equatorial guinea', 'gabon', 'são tomé & principe', 'sao tome and principe']

In [153]:
def check_africa(country_series):
  country_df = country_series.to_frame()
  is_africa = False
  final_contry = None

  for idx, row in country_df.iterrows():
    if row['country'] in northern_countries:
      is_africa = True
      final_country = 'Northern Africa'
      break
    elif row['country'].lower() in southern_countries:
      is_africa = True
      final_country = 'Southern Africa'
      break
    elif row['country'].lower() in west_countries:
      is_africa = True
      final_country = 'Western Africa'
      break
    elif row['country'].lower() in east_countries:
      is_africa = True
      final_country = 'Eastern Africa'
      break
    elif row['country'].lower() in central_countries:
      is_africa = True
      final_country = 'Central Africa'
      break
    else:
      final_country = row['country']

  return is_africa, final_country

def check_african_authors_pos(authors_info, authors_ord_info, eid):
  authors_list = authors_ord_info["authors_list"]
  
  authors_parsed_order = []

  for idx, _ in enumerate(authors_list):
    order = 'inbetween'
    if idx == 0:
      order = 'first'
    elif idx == len(authors_list)-1: 
      order = 'last'

    print(authors_list.loc[idx])
    affiliations = authors_list.loc[idx].split(',')[4].split('=')[1].split(')')[0].split(';')
    affiliations[0] = affiliations[0][1:] # remove extra quote
    affiliations[-1] = affiliations[-1][:-1]

    index_name = authors_list.loc[idx].split(',')[1].split('=')[1].strip()

    print(affiliations, ' ', index_name)
    author_info = []
    for aff in affiliations:
      author_info.append(authors_info[authors_info["affiliation_id"] == float(aff)])
    
    author_info = pd.concat(author_info, axis=0)
    is_africa, final_country = check_africa(author_info.country)

    print(is_africa, ' ', order, ' ', final_country)
    authors_parsed_order.append(pd.DataFrame.from_dict({'order': order}))
  
  authors_parsed_order = pd.concat(authors_parsed_order, axis=0)
  return authors_parsed_order

In [154]:
# 

for idx, row in pubs.iterrows():
  print('EID- ', row['eid'])
  authors_info = authors[authors['eid']==row['eid']]
  authors_ord_info = authors_order[authors_order['eid']==row['eid']]

  if len(authors_ord_info["authors_list"]) == 0:
    print('Publication doesnt have authors order : EID- ', row['eid'])
    continue

  check_african_authors_pos(authors_info, authors_ord_info, row['eid'])
  break

EID-  2-s2.0-85127682728
Publication doesnt have authors order : EID-  2-s2.0-85127682728
EID-  2-s2.0-85141298749
Author(auid=57217148607, indexed_name='Adam J.M.', surname='Adam', given_name='Jibril Muhammad', affiliation='60018205;60105028')
['60018205', '60105028']   'Adam J.M.'
True   first   Western Africa


ValueError: ignored